In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from json import load


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)
with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [ ]:
for event in discription_of_event:
    

In [3]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(d)
        dict_of_disc['is_discr'].append(True)



In [4]:
df = pd.DataFrame(dict_of_disc)
df

,expession,is_discr
0,основные положения теории истории,True
1,основные положения методологии истории,True
2,место истории в системе гуманитарного знания,True
3,закономерности исторического процесса,True
4,этапы исторического процесса,True
...,...,...
702,методикой использования основных численных мет...,True
703,методикой использования основных численных мет...,True
704,методикой использования основных численных мет...,True
705,методикой использования численных методов реше...,True


In [5]:
# model_class, tokenizer_class, pretrained_weights = (
#     ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Хотите BERT вместо distilBERT? Раскомментируйте следующую строку:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Загрузка предобученной модели/токенизатора
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
tokenized = df['expession'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
padded = [tokenizer.encode(i, add_special_tokens=True) for i in df['expession'].to_list()]
ml = len(max(padded, key=len))
for i in padded:
    for _ in range(ml-len(i)):
        i.append(0)

padded = np.array(padded)

array([[  101,  1193, 29747, ...,     0,     0,     0],
       [  101,  1193, 29747, ...,     0,     0,     0],
       [  101,  1191, 15290, ...,     0,     0,     0],
       ...,
       [  101,  1191, 15290, ...,     0,     0,     0],
       [  101,  1191, 15290, ...,     0,     0,     0],
       [  101,  1192, 10260, ...,     0,     0,     0]])

In [20]:
input_ids = torch.tensor(padded)

with torch.no_grad():
    last_hidden_states = model(input_ids)


In [22]:
features = last_hidden_states[0][:,0,:].numpy()


array([[-0.41903007,  0.66514844,  0.60549873, ..., -0.48144794,
         0.37142098, -0.2614745 ],
       [-0.38664246,  0.6248358 ,  0.6110348 , ..., -0.43220413,
         0.3550335 , -0.26506042],
       [-0.40422782,  0.5560879 ,  0.57891154, ..., -0.41285732,
         0.38763666, -0.31203648],
       ...,
       [-0.13475193,  0.6310348 ,  0.35162398, ..., -0.28626424,
         0.28229105, -0.12481178],
       [-0.12296842,  0.47135338,  0.5591191 , ..., -0.15835932,
         0.34259963, -0.18688935],
       [-0.34440002,  0.4739834 ,  0.57158476, ..., -0.2617385 ,
         0.35737425, -0.2996565 ]], dtype=float32)

In [23]:
labels = df['is_discr']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)


In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
